### Connection with google drive and instaling the necessary Libaries

In [329]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [330]:
import os
os.chdir('/content/drive/MyDrive/resume_parser')

In [ ]:
!pip install transformers

### Custom hardcode some sample data format for resume parsing

In [335]:
import torch
from transformers import BertTokenizer,BertTokenizerFast, BertForTokenClassification
from torch.utils.data import Dataset, DataLoader, RandomSampler
from torch.optim import Adam

# Sample annotated data (replace this with your actual annotated data)
resume_data = [
    {
        "content": "Abhishek Jha\nApplication Development Associate - Accenture\n\nBengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a\n\n• To work for an organization which provides me the opportunity to improve my skills...",
        "annotation": [
            {
                "label": ["Name"],
                "points": [{"start": 0, "end": 12, "text": "Abhishek Jha"}]
            },
            {
                "label": ["Email Address"],
                "points": [{"start": 193, "end": 223, "text": "indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a"}]
            },
            {
                "label": ["Skills"],
                "points": [{"start": 243, "end": 266, "text": "..."}]
            },
            # Add more annotations here...
        ]
    },
    {
        "content": "Jane Smith\nData Scientist - XYZ Corp\n\nNew York, USA - Email me on Indeed: indeed.com/r/Jane-Smith/2345xyz\n\n• Highly skilled data scientist with a passion for analyzing complex datasets and providing valuable insights...",
        "annotation": [
            {
                "label": ["Name"],
                "points": [{"start": 0, "end": 10, "text": "Jane Smith"}]
            },
            {
                "label": ["Email Address"],
                "points": [{"start": 137, "end": 172, "text": "indeed.com/r/Jane-Smith/2345xyz"}]
            },
            {
                "label": ["Skills"],
                "points": [{"start": 192, "end": 221, "text": "..."}]
            },
            # Add more annotations here...
        ]
    },
    {
        "content": "Michael Johnson\nMarketing Manager - ABC Marketing\n\nLos Angeles, California - Email me on Indeed: indeed.com/r/Michael-Johnson/5678abc\n\n• Experienced marketing manager with a proven track record of executing successful marketing campaigns...",
        "annotation": [
            {
                "label": ["Name"],
                "points": [{"start": 0, "end": 15, "text": "Michael Johnson"}]
            },
            {
                "label": ["Email Address"],
                "points": [{"start": 114, "end": 159, "text": "indeed.com/r/Michael-Johnson/5678abc"}]
            },
            {
                "label": ["Skills"],
                "points": [{"start": 181, "end": 216, "text": "..."}]
            },
            # Add more annotations here...
        ]
    },
    {
        "content": "Emily Wang\nSoftware Engineer - XYZ Tech\n\nSan Francisco, California - Email me on Indeed: indeed.com/r/Emily-Wang/8765xyz\n\n• Skilled software engineer with expertise in full-stack web development and building scalable applications...",
        "annotation": [
            {
                "label": ["Name"],
                "points": [{"start": 0, "end": 11, "text": "Emily Wang"}]
            },
            {
                "label": ["Email Address"],
                "points": [{"start": 122, "end": 159, "text": "indeed.com/r/Emily-Wang/8765xyz"}]
            },
            {
                "label": ["Skills"],
                "points": [{"start": 177, "end": 221, "text": "..."}]
            },
            # Add more annotations here...
        ]
    },
    # Add more resumes here...
]






In [336]:
print(resume_data)

[{'content': 'Abhishek Jha\nApplication Development Associate - Accenture\n\nBengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a\n\n• To work for an organization which provides me the opportunity to improve my skills...', 'annotation': [{'label': ['Name'], 'points': [{'start': 0, 'end': 12, 'text': 'Abhishek Jha'}]}, {'label': ['Email Address'], 'points': [{'start': 193, 'end': 223, 'text': 'indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a'}]}, {'label': ['Skills'], 'points': [{'start': 243, 'end': 266, 'text': '...'}]}]}, {'content': 'Jane Smith\nData Scientist - XYZ Corp\n\nNew York, USA - Email me on Indeed: indeed.com/r/Jane-Smith/2345xyz\n\n• Highly skilled data scientist with a passion for analyzing complex datasets and providing valuable insights...', 'annotation': [{'label': ['Name'], 'points': [{'start': 0, 'end': 10, 'text': 'Jane Smith'}]}, {'label': ['Email Address'], 'points': [{'start': 137, 'end': 172, 'text': 'indeed.com/r/Jane-Smith/2345xyz'

### Reading the JSON format of your annotated Data

In [337]:
import json

# Read the text file and split it into separate JSON objects
with open('cvs_annotation_test.txt', 'r') as file:
    json_data = file.read().split('\n}\n')
json_data = [json_obj + '}' for json_obj in json_data[:-1]]
i=0
# data=[]
# for dat in json_data:
#   data.append(json_data)
# print(json_data[-1])
all_data=[]
new_data=[]
for i in range(len(json_data)):
  try:
    one_val = json.loads(json_data[i])
    all_data.append(one_val)
  except:
    try:
      # print(i)
      # Find the position of "annotation" key
      annotation_index = json_data[i].find('"annotation"')
      # Add a comma before "annotation" key
      fixed_json_data = json_data[i][:annotation_index - 1] + "," + json_data[i][annotation_index:]
      one_val = json.loads(fixed_json_data)
      new_json_data = {
        "content": one_val["content"],
        "annotation": one_val["annotation"],
        "extra": None
    }
      all_data.append(new_json_data)
    except:
      print(i)


# Print the new JSON string
print(all_data)


6
[{'content': 'Ayesha Khalique Hiring Manager Trey Research 89 Pacific Ave San Francisco, CA 65432 Dear Ayesha Khalique, Want to put your own image in the circle? It is easy! Select the image and do a right mouse click. Select Fill from the shortcut menu. Choose Picture from the list. Navigate your computer to get the appropriate picture. Click okay to insert your selected image. Once your image has been inserted, select it again. Go to the Picture Tools Format menu. Click on the down arrow below Crop and select Fill from the list. This will auto adjust your image to crop to the image. You can click and drag your image to place it appropriately. Are you looking for a job title with: Number years of hands-on experience in the area of expertise? Knowledge of the latest technology in the industry or field? Excellent written and oral communication skills? A passion to learn and to increase his skills? If so, then you need look no further. You will see from my enclosed resume that I meet a

In [338]:
def update_annotation_content(data):
    content = data["content"]
    annotations = data["annotation"]
    labels_to_remove = []  # Store the labels to remove from the data
    updated_annotations = []  # Store the updated annotations

    for ann in annotations:
        points = ann["points"]
        updated_points = []
        for point in points:
            text = point["text"]
            try:
                start = content.index(text)
                end = start + len(text)
                updated_points.append({
                    "start": start,
                    "end": end,
                    "text": text
                })
            except ValueError:
                labels_to_remove.append(ann)
                break

        if updated_points:
            updated_annotations.append({
                "label": ann["label"],
                "points": updated_points
            })

    # Update the annotations in the data dictionary
    data["annotation"] = updated_annotations

# Update the start and end values in the data dictionary and remove problematic labels
new_all_data = []
for data in all_data:
    update_annotation_content(data)
    new_all_data.append(data)

# Print the updated data
for data in new_all_data:
    print(data)
    break


{'content': 'Ayesha Khalique Hiring Manager Trey Research 89 Pacific Ave San Francisco, CA 65432 Dear Ayesha Khalique, Want to put your own image in the circle? It is easy! Select the image and do a right mouse click. Select Fill from the shortcut menu. Choose Picture from the list. Navigate your computer to get the appropriate picture. Click okay to insert your selected image. Once your image has been inserted, select it again. Go to the Picture Tools Format menu. Click on the down arrow below Crop and select Fill from the list. This will auto adjust your image to crop to the image. You can click and drag your image to place it appropriately. Are you looking for a job title with: Number years of hands-on experience in the area of expertise? Knowledge of the latest technology in the industry or field? Excellent written and oral communication skills? A passion to learn and to increase his skills? If so, then you need look no further. You will see from my enclosed resume that I meet all 

In [339]:
for dat in range(len(new_all_data)):
  print(new_all_data[dat])

{'content': 'Ayesha Khalique Hiring Manager Trey Research 89 Pacific Ave San Francisco, CA 65432 Dear Ayesha Khalique, Want to put your own image in the circle? It is easy! Select the image and do a right mouse click. Select Fill from the shortcut menu. Choose Picture from the list. Navigate your computer to get the appropriate picture. Click okay to insert your selected image. Once your image has been inserted, select it again. Go to the Picture Tools Format menu. Click on the down arrow below Crop and select Fill from the list. This will auto adjust your image to crop to the image. You can click and drag your image to place it appropriately. Are you looking for a job title with: Number years of hands-on experience in the area of expertise? Knowledge of the latest technology in the industry or field? Excellent written and oral communication skills? A passion to learn and to increase his skills? If so, then you need look no further. You will see from my enclosed resume that I meet all 

In [359]:
resume_data=new_all_data

### Preprocessing of data to make it suitable for the model

In [361]:

def get_label(offset, labels):
    print("offset:",offset[0],",",offset[1])
    if not labels or offset[0]==offset[1]:
        return 'O',(None,None)
    for label in labels:
        label_start = label['points'][0]['start']

        label_end = label['points'][0]['end']
        if label_start <= offset[0] and offset[1] < label_end:
            print(label_start,",",label_end)

            print(label['label'])
            return label['label'],(label_start,label_end)
    return 'O',(None,None)


tags_vals = ["O","Objective","Experience", "LinkedIn","Career Goals","Profile","Greeting","Greeting","Company Name","Job Title","Phone No.", "Website","Name","Address","Email","Companies worked at", "Phone No","Skills", "Email Address",
             "Location","Degree","Skills","College Name","Graduation Year","Education","Designation","Years of Experience"]

tag2idx = {t: i for i, t in enumerate(tags_vals)}
idx2tag = {i: t for i, t in enumerate(tags_vals)}

In [362]:
class ResumeDataset(Dataset):
    def __init__(self, resume, tokenizer, tag2idx, max_len, is_test=False):
        self.resume = resume
        self.tokenizer = tokenizer
        self.is_test = is_test
        self.tag2idx = tag2idx
        self.max_len = max_len

    def __len__(self):
        return len(self.resume)

    def __getitem__(self, idx):
        data = process_resume(
            self.resume[idx], self.tokenizer, self.tag2idx, self.max_len, self.is_test)
        return {
            'input_ids': torch.tensor(data['input_ids'], dtype=torch.long),
            'token_type_ids': torch.tensor(data['token_type_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(data['attention_mask'], dtype=torch.long),
            'labels': torch.tensor(data['labels'], dtype=torch.long),
            'orig_label': data['orig_labels']
        }

In [363]:
import torch
from transformers import BertTokenizerFast, BertForTokenClassification
from torch.utils.data import Dataset, DataLoader, RandomSampler
from torch.optim import AdamW
from tqdm import tqdm
def get_label(offset, labels):
    if offset[0] == 0 and offset[1] == 0:
        return 'O'
    for label in labels:
        if offset[1] >= label[0] and offset[0] <= label[1]:
            return label[2]
    return 'O'
def process_resume(data, tokenizer, tag2idx, max_len, is_test=False):
    tok = tokenizer.encode_plus(
        data[0], max_length=max_len, return_offsets_mapping=True)
    curr_sent = {'orig_labels': [], 'labels': []}

    padding_length = max_len - len(tok['input_ids'])

    if not is_test:
        labels = data[1]['entities']
        labels.reverse()
        for off in tok['offset_mapping']:
            label = get_label(off, labels)
            curr_sent['orig_labels'].append(label)
            curr_sent['labels'].append(tag2idx[label])
        curr_sent['labels'] = curr_sent['labels'] + ([0] * padding_length)

    curr_sent['input_ids'] = tok['input_ids'] + ([0] * padding_length)
    curr_sent['token_type_ids'] = tok['token_type_ids'] + \
        ([0] * padding_length)
    curr_sent['attention_mask'] = tok['attention_mask'] + \
        ([0] * padding_length)
    return curr_sent



MODEL_NAME = 'bert-base-uncased'
TOKENIZER = BertTokenizerFast.from_pretrained(MODEL_NAME, lowercase=True)
model = BertForTokenClassification.from_pretrained(MODEL_NAME, num_labels=len(tag2idx))

# Create ResumeDataset instance
train_d = ResumeDataset(resume_data, TOKENIZER, tag2idx, max_len=500)

# Define the optimizer and learning rate
optimizer = AdamW(model.parameters(), lr=1e-5)

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
epochs = 20
train_sampler = RandomSampler(train_d)
train_dataloader = DataLoader(train_d, sampler=train_sampler, batch_size=1)
# Iterate through the data loader and print the contents of each batch
for batch in train_dataloader:
    input_ids = batch['input_ids']
    token_type_ids = batch['token_type_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']
    orig_labels = batch['orig_label']

    # Convert input_ids back to tokens
    tokens = TOKENIZER.convert_ids_to_tokens(input_ids[0])

    # # Print the tokenized text and original labels
    # for token, orig_label in zip(tokens, orig_labels):
    #     print(token, " ", orig_label)
    # print("=" * 50)
    break



Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [364]:

def get_label(offset, labels):
    if not labels or offset[0]==offset[1]:
        return 'O',(None,None)
    for label in labels:
        label_start = label['points'][0]['start']
        label_end = label['points'][0]['end']
        if label_start <= offset[0] and offset[1] <= label_end:
            return label['label'][0],(label_start,label_end)
    return 'O',(None,None)


tags_vals = ["O","Date","Languages","Projects","Hobbies","Nationality","Date of birth","Objective","LinkedIn","Career Goals","Profile","Greeting","Greeting","Company Name","Job Title","Phone No.", "Website","Name","Address","Email","Companies worked at", "Phone No","Skills", "Email Address",
             "Location","Degree","Experience","College Name","Graduation Year","Education","Designation","Years of Experience"]

tag2idx = {t: i for i, t in enumerate(tags_vals)}
idx2tag = {i: t for i, t in enumerate(tags_vals)}
import torch
from transformers import BertTokenizerFast, BertForTokenClassification
from torch.utils.data import Dataset, DataLoader, RandomSampler
from torch.optim import AdamW
from tqdm import tqdm
def process_resume(data, tokenizer, tag2idx, max_len, is_test=False):
    content = data["content"]
    tok = tokenizer.encode_plus(content, max_length=max_len, return_offsets_mapping=True)
    curr_sent = {'orig_labels': [], 'labels': []}
    padding_length = max_len - len(tok['input_ids'])
    if not is_test:
        labels = data["annotation"]
        # Print the tokens and corresponding offsets to check for misalignments
        # print("Tokens:", tokenizer.convert_ids_to_tokens(tok['input_ids']))
        # print("Offsets:", tok['offset_mapping'])

        # print(labels)
        for off in tok['offset_mapping']:
            start,end=off
            label,new_off = get_label(off, labels)
            start,end=new_off
            if label not in tag2idx:  # Check if label exists in tag2idx, otherwise set it to 'O'
                label = 'O'
            curr_sent['orig_labels'].append(label)
            curr_sent['labels'].append(tag2idx[label])

        curr_sent['labels'] = curr_sent['labels'] + ([0] * padding_length)

        # Print the labels assigned to each token
        # print("Token Labels:", curr_sent['orig_labels'])

    curr_sent['input_ids'] = tok['input_ids'] + ([0] * padding_length)
    curr_sent['token_type_ids'] = tok['token_type_ids'] + ([0] * padding_length)
    curr_sent['attention_mask'] = tok['attention_mask'] + ([0] * padding_length)
    return curr_sent

MODEL_NAME = 'bert-base-uncased'
TOKENIZER = BertTokenizerFast.from_pretrained(MODEL_NAME, lowercase=True)
model = BertForTokenClassification.from_pretrained(MODEL_NAME, num_labels=len(tag2idx))

# Create ResumeDataset instance
train_d = ResumeDataset(resume_data, TOKENIZER, tag2idx, max_len=128)

# Define the optimizer and learning rate
optimizer = AdamW(model.parameters(), lr=1e-5)

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
epochs = 20
train_sampler = RandomSampler(train_d)
train_dataloader = DataLoader(train_d, sampler=train_sampler, batch_size=1)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Model Training

In [1]:
import random

# Assuming 'resume_data' contains the list of resume samples
# Split data into train and validation sets
random.shuffle(resume_data)
train_size = int(0.8 * len(resume_data))  # 80% for training
train_data = resume_data[:train_size]
val_data = resume_data[train_size:]

# Create Dataset instances for train and validation
train_dataset = ResumeDataset(train_data, TOKENIZER, tag2idx, max_len=500)
val_dataset = ResumeDataset(val_data, TOKENIZER, tag2idx, max_len=500)

# Create DataLoaders for train and validation
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=1)

val_sampler = RandomSampler(val_dataset)
val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=1)

# Initialize the optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
epochs =5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    total_correct_predictions = 0
    total_predictions = 0

    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1} - Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Calculate accuracy for this batch
        _, predicted_labels = torch.max(outputs.logits, dim=2)
        correct_predictions = torch.eq(predicted_labels, labels).sum().item()
        total_correct_predictions += correct_predictions
        total_predictions += labels.numel()

        # Backpropagation and optimization
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        optimizer.zero_grad()

    # Calculate average loss and accuracy for this epoch
    avg_loss = total_loss / len(train_dataloader)
    accuracy = total_correct_predictions / total_predictions

    print(f"Epoch {epoch+1} - Training Loss: {avg_loss:.4f}, Training Accuracy: {accuracy:.4f}")

    # Validation loop
    model.eval()
    total_val_loss = 0
    total_val_correct_predictions = 0
    total_val_predictions = 0

    for batch in tqdm(val_dataloader, desc=f"Epoch {epoch+1} - Validation"):
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels=labels)
            val_loss = outputs.loss
            total_val_loss += val_loss.item()

            # Calculate accuracy for this batch
            _, val_predicted_labels = torch.max(outputs.logits, dim=2)
            val_correct_predictions = torch.eq(val_predicted_labels, labels).sum().item()
            total_val_correct_predictions += val_correct_predictions
            total_val_predictions += labels.numel()

    # Calculate average validation loss and accuracy for this epoch
    avg_val_loss = total_val_loss / len(val_dataloader)
    val_accuracy = total_val_correct_predictions / total_val_predictions

    print(f"Epoch {epoch+1} - Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

# Save the trained model
torch.save(
    {
        "model_state_dict": model.state_dict()
    },
    'ner_model.pth',
)


### Testing the Model on Resume text

In [255]:
import torch
from transformers import BertTokenizerFast, BertForTokenClassification

# Initialize the tokenizer and model
MODEL_NAME = 'bert-base-uncased'
TOKENIZER = BertTokenizerFast.from_pretrained(MODEL_NAME, lowercase=True)
model = BertForTokenClassification.from_pretrained(MODEL_NAME, num_labels=len(tag2idx))

# Load the trained model checkpoint
checkpoint_path = 'ner_model.pth'
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])

# Set the model to evaluation mode
model.eval()


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [256]:
def preprocess_single_resume(resume_content, tokenizer, max_len):
    tok = tokenizer.encode_plus(resume_content, max_length=max_len, return_offsets_mapping=True)
    input_ids = tok['input_ids']
    attention_mask = tok['attention_mask']
    token_type_ids = tok['token_type_ids']

    # Padding
    padding_length = max_len - len(input_ids)
    input_ids += [0] * padding_length
    attention_mask += [0] * padding_length
    token_type_ids += [0] * padding_length

    return {
        'input_ids': torch.tensor(input_ids, dtype=torch.long).unsqueeze(0),
        'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long).unsqueeze(0),
        'attention_mask': torch.tensor(attention_mask, dtype=torch.long).unsqueeze(0)
    }


In [257]:
def get_predicted_labels(model_output, idx2tag):
    _, predicted_labels = torch.max(model_output, dim=2)
    predicted_labels = predicted_labels.squeeze(0).tolist()
    predicted_labels = [idx2tag[label_idx] for label_idx in predicted_labels]
    return predicted_labels

def predict_resume_labels(resume_content, model, tokenizer, idx2tag, max_len):
    input_data = preprocess_single_resume(resume_content, tokenizer, max_len)
    input_ids = input_data['input_ids']
    token_type_ids = input_data['token_type_ids']
    attention_mask = input_data['attention_mask']

    with torch.no_grad():
        model_output = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)

    predicted_labels = get_predicted_labels(model_output.logits, idx2tag)
    return predicted_labels


In [258]:
resume_content = "Abhishek Jha\nApplication Development Associate - Accenture\n\nBengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a\n\n• To work for an organization which provides me the opportunity to improve my skills"
input_data = preprocess_single_resume(resume_content, TOKENIZER, max_len=500)
print("Input Data:", input_data)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Input Data: {'input_ids': tensor([[  101, 11113, 24158,  5369,  2243,  1046,  3270,  4646,  2458,  5482,
          1011,  9669,  5397,  8191, 14129,  1010, 12092,  1011, 10373,  2033,
          2006,  5262,  1024,  5262,  1012,  4012,  1013,  1054,  1013, 11113,
         24158,  5369,  2243,  1011,  1046,  3270,  1013,  2184,  2063,  2581,
          2050,  2620, 27421,  2581, 16703,  9818, 23777,  2050,  1528,  2000,
          2147,  2005,  2019,  3029,  2029,  3640,  2033,  1996,  4495,  2000,
          5335,  2026,  4813,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,    

In [273]:
from transformers import BertTokenizerFast

MODEL_NAME = 'bert-base-uncased'
TOKENIZER = BertTokenizerFast.from_pretrained(MODEL_NAME, lowercase=True)

resume_content = "Abhishek Jha\nApplication Development Associate - Accenture\n\nBengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a\n\n• To work for an organization which provides me the opportunity to improve my skills"
resume_content="""AYAZ MEHMOOD Rawalpindi , Pakistan  92-321-9872721  ayazmehmood550gmail.com  https://www.linkedin.com/in/ayaz -mehmood -5b941219a/  https://github.com/ayazmhmd Experience d AI E ngineer with machine learning , Statistical, Analytical and programming skills seeking employment with an innovative Data Science/AI firm to expand my skill set and software programming proficiency. PROFESSIONAL EXPERIENCE Medical Imaging and Diagnostic Lab (NCAI) Oct 2022  Present Research Assistant  Working on different project s of Medical Imaging (Breast Cancer, TB, Brain tumor)  Devised AI based solution (Dicom Viewer) as a assistive tools for Doctors  Tried Different AI Models on Med ical image classification, detection and segmentation.  Organized different Event rel ated to Arti ficial Intelligence including Seminars, Workshop and presentation. National Center for Cyber Security April 2022 Oct 2022 AI Research Scholar  Designed and tested new AI systems for Industrial applications.  Coordinated with other engineers and programmers to meet deadlines of our weekly work progress.  Devised AI based System for real time attendance mark using face recognition.  Assisted MS
research scholars in their Research projects. Freelancing 2020  Present Data Analyst and ML Engineer  Collaborated with my clients to design, create, test and deploy Artificial Intelligence Models  Worked alongside a team of AI to develop, test and deploy a model that effectively detect fire from a video stream in real time and send notif ication message with location and exact time .  Worked on other specifically computer vision projects related to image classification and detection and APIs creation.  Successfully deployed multiple Artificial Intelligence projects on Hureko and AWS Cloud EDUCATION University of Engineering and Technology Peshawar , Pakistan 2018-2022 Bachelor of Computer Systems Engineering Major: Computer CGPA (3.74 ) Thesis : Anomaly Detection from Video Streams Using Deep Learning techniques . PROJECTS ML and DL Projects:  Completed regression, classification, clustering and time series projects in Python using Scikit - Learn. Codes are available here on the GitHub.  Implemented CNN on images and videos. Good knowledge of Image segmentation, classification and Detection. Experience in NLP for sentiment analysis and text classification . All c odes are available here on the GitHub.  Experience in Kaggle competition in Natural language processing and forecasting. Anomaly Detection from Video Streams Using Deep Learning techni ques (FYP):  Set up a system that was efficient enough to Detect and classify unusual ac tivities from video streams.  Worked on 3 Anomalous categories (Fighting, Shoplifting and Explosion) and achieved accuracy of 82 .  Developed a flutter application as a use case of our project that uses real time camera for video streams  Served the flask API on Hureko Server. Details o f the project are available here Arrhythmia Detection Using Deep Learning Techniques (Research Project):  Classified ECG signal using a Conventional Neural Netw ork (CNN)  LSTM.  Submitted the paper to IEEE Transactions on Biomedical circuits and systems. Codes are available here on the GitHub. Multimedia Analysis of Disaster -Related Social Media Data (Research Project):  In the fi rst part, we Classified the twit ter data as disaster or not from the data shared by the organization  In the second part, we specified the location using named entity recognition  We have used different modeling technique as the project has two parts. We explored the state-of-the-art model for this and achieved F1 score of 92 .  Presented our paper in mediaeval competition. Data Analysis:  Conducted exploratory data analysis (EDA) on various datasets using Python and data visualization libraries such as Matplotlib, Seaborn, and Plotly.  Utilized statistical methods and visualization techniques to identify patterns, trends, and outliers in the data, leading to insights that informed business decisions.  Developed custom scripts and tools to automate data cle aning and preprocessing tasks, improving the efficiency and accuracy of the EDA process.  Worked on PowerBI dashboard for practice and for one of my clients.  Codes are available in my GitHub repositories link. Early Detection of Alzheimer's disease using vision Transformer :  Developed a custom vision transformer Model for Alzheimers disease classification.  Fine -tuned the model and achieved accuracy of 9 6. Code s are available in this link.  Submitted a paper to a journal by adding some computer vision Model results Face  Eye tracking using OpenCV :  Developed a model that detect and track a person face and eyes continuously by laptop Camera.  The Model create a buzzer sound when no face is detected. Code s are available in this link. Yolov5 and Yolov8 Object detection Projects:  Developed and trained multiple YOLOv8 models for object detection, including gun detection, pill detection, car detection, and football player detection.  Utilized various dee p learning techniques to preprocess and augment the training data, resulting in highly accurate and robust object detection models.  Created a Flask API to deploy the object detection models and enable real -time detection of objects in images and videos.  Conducted thorough testing and validation of the object detection models, including precision and recall analysis, to ensure high performance and reliability in different environments and scenarios.  Developed custom data annotation tools and pipelines to str eamline the data labeling and preprocessing process, improving the efficiency and quality of the training data  Codes are available in my
GitHub repositories link. Smart -attendance system using face recognition:  Designed and tested a system that recognizes faces and mark attendance.  System saved the recognized faces in an excel sheet. Code s are available in this link. Chatbot in python:  Developed a chatbot using Python and utilized a Kaggle dataset to train the natural language processing model.  Conducted thorough data analysis on Kaggle datasets to improve the chatbot's accuracy, leading to a 50 reduction in support inquiries. Model APIs and Deployment:  Served Flask ML and DL API s on Hureko Server .  Created Flask API for Multiple projects of cla ssification, segmentation and detection.  Integrated different models into flutter app by using tflite and also connected with Flask API.  Deployed of ML Model on AWS Server (Amazon Sage Maker ). Web Scrapping:  Scraped different websites (Twitter
mostly) using Scrapy, beautiful soup, and Parse hub .  Scraped product data across different domains for analysis from online shopping websites .
Database Design :  Designed and developed an online resume builder using PHP and SQL database .  Product management system for medical store for a client using Laravel and SQL database .  Flask Web projects and deployment of ML and DL model. Semester Projects:  Space shooter game in python (4 Level).  TIC TAC TOE game and Console Based online delivery system in C.  Automatic Street Light using PIR Sensor, Fire Alarm, Firefighting Robot.  Hotel, School and Hospital Manageme nt System using Linked list in data structure .  Created custom Unix C Shell and also Interprocess communication using messa ge Queue Method, named pipe and Shared Memory in Unix C. CERTIFICATIONS  Programming for Everybody (Getting Started with Python) - Coursera  Introduction to Artificial Intelligence  Coursera  Neural Network  Deep learning  Coursera  Introduction to Data Science in Python
Coursera  Applied Machine Learning in Python  Coursera  Programming Basics Great Learning  Introduction to Machine Learning -Great Learning  Python for Machine Learning - Great Learning  Deep Learning, Computer vision - Great Learning SKILLS  Technical Skills Python, Data analytics, Statistical analysis, Machine Learning, Deep Learning, Comp uter vision, NLP, YOLO, Web scraping , Model Deployment, Model integration, Hureko, Amazon SageMaker , AWS, Flask, PowerBI, Excel, SQL Database.  Soft Skills Self-Learning, Problem Solving, Critical thinking , Interpersonal Skill
, Teamwork, Leadership, Communication."""
resume_content="""AZIZ UR RAHMAN STUDENT PROFILE A Highly motivated Software Engineering student with a passion for web development and mobile app development. Skilled in HTML, CSS and little bit Java Script, currently expanding proficiency in Dart language and Flutter. Committed to delivering high - quality projects and continuously learning and adapting new technologies. CONTACT PHONE: 0318 9065233 LOCATION: Ringroad Peshawar, Pakistan EMAIL: azixkhan.55@gmail.com MOTIVATION Passionate about continuous learning. Seeking an opportunity to collaborate with professionals who can inspire and mentor me, while bringing my own skills to contribute to your teams achievements. EDUCATION City University 2021 - 2025 Studying Software Engineering (In Progress) from city university of science and information technology. PROJECTS AND PORTFOLIO Pharmacy Management System Basic Website: Developed a basic website for a pharmacy management system. Utilized HTML, CSS, and JavaScript to create an interactive user interface. Restaurant Management System: Developed a web -based system for restaurant management. Implemented features such as menu management, order processing, and reservation tracking. Utilized HTML, CSS, JavaScript, and backend technologies for data management. Various Blogs: Created and maintained different blogs using Flask and Python. Wrote articles on technology, software development, and personal experiences. SKILLS 8045807080In ProgressIn Progress 0.00 25.00 50.00 75.00 100.00CSSJava ScriptPythonFlaskHTMLDartFlutter",
"""
# Tokenize the resume content using the provided tokenizer
tokenized_resume = TOKENIZER.encode_plus(resume_content, max_length=500, return_offsets_mapping=True)

# Convert token IDs to tokens
resume_tokens = TOKENIZER.convert_ids_to_tokens(tokenized_resume['input_ids'])
print(resume_content)
# Display the tokens
print(resume_tokens)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


AZIZ UR RAHMAN STUDENT PROFILE A Highly motivated Software Engineering student with a passion for web development and mobile app development. Skilled in HTML, CSS and little bit Java Script, currently expanding proficiency in Dart language and Flutter. Committed to delivering high - quality projects and continuously learning and adapting new technologies. CONTACT PHONE: 0318 9065233 LOCATION: Ringroad Peshawar, Pakistan EMAIL: azixkhan.55@gmail.com MOTIVATION Passionate about continuous learning. Seeking an opportunity to collaborate with professionals who can inspire and mentor me, while bringing my own skills to contribute to your teams achievements. EDUCATION City University 2021 - 2025 Studying Software Engineering (In Progress) from city university of science and information technology. PROJECTS AND PORTFOLIO Pharmacy Management System Basic Website: Developed a basic website for a pharmacy management system. Utilized HTML, CSS, and JavaScript to create an interactive user interfa

In [274]:
print(len(resume_tokens))

293


In [275]:
predicted_labels = predict_resume_labels(resume_content, model, TOKENIZER, idx2tag, max_len=500)
print("Predicted Labels:", predicted_labels)


Predicted Labels: ['O', 'Name', 'Name', 'Name', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Phone No', 'Phone No', 'Phone No', 'Phone No', 'Phone No', 'Phone No', 'O', 'O', 'Address', 'Address', 'Address', 'Address', 'Address', 'Address', 'O', 'O', 'Email', 'Email', 'Email', 'Email', 'Email', 'Email', 'Email', 'Email', 'Email', 'Email', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

### Result

In [276]:
# Define reverse mapping for integer labels to tag names
idx2tag = {i: t for i, t in enumerate(tags_vals)}

# Initialize variables to store entity spans and information
entities = []
current_entity = None
start_position = None

# Loop through the predicted labels to identify entity spans
for idx, label in enumerate(predicted_labels):
    if label == "O":
        # If it's an "O" label and there's a current entity, save it
        if current_entity is not None:
            entities.append((current_entity, start_position, idx - 1))
            current_entity = None
            start_position = None
    else:
        # Beginning of a new entity or continuation of the current entity
        if current_entity is not None and label == current_entity:
            continue  # Continue with the current entity type
        elif current_entity is not None:
            entities.append((current_entity, start_position, idx - 1))
        current_entity = label
        start_position = idx

# Check if there's an entity at the end of the sequence
if current_entity is not None:
    entities.append((current_entity, start_position, len(predicted_labels) - 1))

# Extract and display entity information
for entity, start, end in entities:
    entity_text = " ".join(resume_tokens[start:end + 1])
    print(f"Entity Type: {entity}, Text: {entity_text}")


Entity Type: Name, Text: aziz ur rahman
Entity Type: Phone No, Text: 03 ##18 90 ##65 ##23 ##3
Entity Type: Address, Text: ring ##ro ##ad peshawar , pakistan
Entity Type: Email, Text: az ##ix ##khan . 55 @ gma ##il . com


In [277]:
# Define reverse mapping for integer labels to tag names
idx2tag = {i: t for i, t in enumerate(tags_vals)}

# Initialize variables to store entity spans and information
entities = []
current_entity = None
start_position = None

# Loop through the predicted labels to identify entity spans
for idx, label in enumerate(predicted_labels):
    if label == "O":
        # If it's an "O" label and there's a current entity, save it
        if current_entity is not None:
            entities.append((current_entity, start_position, idx - 1))
            current_entity = None
            start_position = None
    else:
        # Beginning of a new entity or continuation of the current entity
        if "-" in label:
            entity_type = label.split("-")[1]
        else:
            entity_type = label
        if current_entity is not None and entity_type == current_entity:
            continue  # Continue with the current entity type
        elif current_entity is not None:
            entities.append((current_entity, start_position, idx - 1))
        current_entity = entity_type
        start_position = idx

# Check if there's an entity at the end of the sequence
if current_entity is not None:
    entities.append((current_entity, start_position, len(predicted_labels) - 1))

# Extract and display entity information
for entity, start, end in entities:
    entity_tokens = tokenized_resume['input_ids'][start:end + 1]
    entity_text = TOKENIZER.decode(entity_tokens, skip_special_tokens=True)
    print(f"Entity Type: {entity}, Text: {entity_text}")


Entity Type: Name, Text: aziz ur rahman
Entity Type: Phone No, Text: 0318 9065233
Entity Type: Address, Text: ringroad peshawar, pakistan
Entity Type: Email, Text: azixkhan. 55 @ gmail. com
